In [87]:
import os

import geopandas as gpd
import numpy as np
import rasterio
from rasterio import windows
from rasterio.features import geometry_window
from shapely.geometry import mapping

In [81]:
os.environ["GDAL_PAM_ENABLED"] = "NO"

In [84]:
def get_windows(xmin, ymin, xmax, ymax, tile_width, tile_height, overlap):
    xstep = tile_width - overlap
    ystep = tile_height - overlap
    for x in range(xmin, xmax, xstep):
        if x + tile_width > xmax:
            x = xmax - tile_width
        for y in range(ymin, ymax, ystep):
            if y + tile_height > ymax:
                y = ymax - tile_height
            window = windows.Window(x, y, tile_width, tile_height)
            yield window

In [91]:
tile_width = 512
tile_height = 512
tile_overlap = 256

filepath = "data/images/uavforsat_DDH001_ortho_8857.tif"
out_path = "data/tiles/"
label_path = "data/labels"
out_filename = "{}_{}_{}.tif"

metadata = src.meta.copy()
filename = os.path.basename(filepath)

label_filename = filename.replace("_8857.tif", "_polygons.gpkg")
gdf_label = gpd.read_file(os.path.join(label_path, label_filename), layer="aoi")
gdf_label = gdf_label.to_crs("EPSG:8857")

with rasterio.open(filepath) as src:
    metadata = src.meta.copy()

    for _, row in gdf_label.iterrows():
        aoi_window = geometry_window(src, [row.geometry])
        xmin, ymin = aoi_window.col_off, aoi_window.row_off
        xmax, ymax = xmin + aoi_window.width, ymin + aoi_window.height

        for window in get_windows(
            xmin, ymin, xmax, ymax, tile_width, tile_height, tile_overlap
        ):
            transform = windows.transform(window, src.transform)
            metadata["transform"] = transform
            metadata["width"], metadata["height"] = window.width, window.height
            out_filepath = os.path.join(
                out_path,
                out_filename.format(filename[:-4], window.col_off, window.row_off),
            )
            out_image = src.read(window=window)
            if np.count_nonzero(out_image == 255) / out_image.size < 0.01:
                with rasterio.open(out_filepath, "w", **metadata) as dst:
                    dst.write(out_image)